# Set Up

In [ ]:
import api_wandb
import data_processing
import environment
import executions
import main
import utils

Set the path to the [FMA](https://github.com/mdeff/fma?tab=readme-ov-file) data directory. This directory should contain two folders; a folder called `fma_large` obtained from [https://os.unil.cloud.switch.ch/fma/fma_large.zip](https://os.unil.cloud.switch.ch/fma/fma_large.zip) and another folder called `fma_metadata` obtained from [https://os.unil.cloud.switch.ch/fma/fma_metadata.zip](https://os.unil.cloud.switch.ch/fma/fma_metadata.zip).
In the machines we used from the Guacamole server with the user `EDXN01` the path was : `"/home/datasets/FreeMusicArchive"`, change yours accordingly here:

In [ ]:
environment.DATA_PATH = "/home/datasets/FreeMusicArchive"
print(f'Data path set up to: "{environment.DATA_PATH}"')

To test and debug the code this program can be ran in two modes:
- `INFO` which will deactivate debug funtionalities and ran the code with the whole dataset.
- `DEBUG` wich will activate debug functionalities and ran the code with only 100 random samples of the dataset.

We run it in `INFO` level, if you wish to run it in `DEBUG` change it here:

In [ ]:
environment.LOG_LEVEL = environment.logging.INFO

And we set up the Weights & Biases logging (wandb). If you do not want to use wandb set `environment.WANDB_LOGGING` to `False`.

In [ ]:
environment.WANDB_LOGGING = True # Set it to False to disable wandb logging
if environment.WANDB_LOGGING:
    environment.os.environ["WANDB_NOTEBOOK_NAME"] = "final_submission_main.ipynb"

For our project we used our entity `"grup-1-Xarxes-Neuronals-i-Aprenentatge-Profund"` and project `"fma-genre-classification"`, repalce this with your team and project name found at [https://wandb.ai/](https://wandb.ai/) so you can access the results.

In [ ]:
if environment.WANDB_LOGGING:
    api_wandb.ENTITY = "grup-1-Xarxes-Neuronals-i-Aprenentatge-Profund"
    api_wandb.PROJECT = "fma-genre-classification"
    print(f'WANDB Entity set up to: {api_wandb.ENTITY}')
    print(f'WANDB Project set up to: {api_wandb.PROJECT}')

Logging on your Weight & Bias account using your API key found at [https://wandb.ai/authorize](https://wandb.ai/authorize).
If `environment.WANDB_LOGGING` set to `False` it will not prompt you to log in using your wandb API key.

In [ ]:
if environment.WANDB_LOGGING:
    environment.wandb.login(relogin=True, force=True, verify=True)

This variable controls if when an execution of the run fails it continues to the next one printing the error (set to `True`) or it raises the error and stops the Notebook (set to `False`).

In [ ]:
executions.NEXT_WHEN_ERROR = False

We initialize the environment:

In [ ]:
environment.init()

# Verify data is downloaded, is not corrupted, can be read correctly and loaded properly
This will verify that all the data downloaded from the FMA dataset is not corrupte,d the sumchecks match and all files are in the correctdirectoryy. It will also load the data and sade the paths into a file for faster executions times. This will take some time the first time executed but from then on will be alssot inmediatly.

In [ ]:
data_processing.get_data()

# Experiments and Executions

Here we have all the experiments and executions we have done with the configuration code to run the `main.main()` on them.

We used to train 100 epochs, with it each execution takes between 5 to 8 hours, if you want to execute something quick to check if it works lower the amount of epochs to 1 or 2.

In [ ]:
executions.NUMBER_EPOCHS = 100

## WARNING 1
The batch size of the execution have been set individually for each model so they use the maximum amount of memory of the available 12288MiB in the GPU we used for training. If you get a `CUDA out of memory` error, reduce the `batch_size` in the settings of the executions `dataloader -> parameters -> batch_size`. 

## WARNING 2
If you change the batch size, since we use `OneCycleLR` as learning scheduler it needs to have the `batch_size` changed too, in the settigns of the execution `learning_rate_schedulers -> OneCycleLR -> parameters -> total_steps`, where `total_steps = number_epochs * ceil(floor(data_samples * percentage_train) / batch_size)`.

## Experiment 1

In this experiment we will try the GRU + classifier models with spectogram as input, trying to classify all genres at the same time.

#### Execution 1

Firstly, we will try it with a hidden dimension of 256.

##### Setup and configuration Experiment 1, Execution 1: hidden dimension 256

In [ ]:
executions.EXECUTIONS = {
    'experiment_1_execution_1_hidden_dim_256': {
        'data_transformation': {
            'name': 'mel_spectrogram_pre_saved',
            'parameters': {
                'sample_rate': 16000,
                'number_frames': 16000 * 29,
                'number_fft_points': 1024,
                'hop_length': 1024,
                'window_length': 512,
                'power': 2.0,
                'number_mel_channels': 128,
                'to_decibels': True,
            },
        },
        'data_split': {
            'name': 'random_split',
            'parameters': {
                'percentage_test': 0.2,
            },
        },
        'dataset':  {
            'name': 'dataset_with_spectograms_pre_saved',
            'parameters': {},
        },
        'dataloader': {
            'name': 'dataloader',
            'parameters': {
                'batch_size': 700,
                'shuffle': True,
                'pin_memory': False,
                'prefetch_factor': 5,
                'persistent_workers': True,
            },
        },
        'weights': {
            'parameters': {
                'method': 'inver_log',
                'epsilon': 1e-6,
            },
        },
        'model': {
            'name': 'baseline',
            'parameters': {
                'hidden_dim': 256,
                'classifier_hidden_layers': [],
                'threshold_method': 'roc_closest',
                'dropout_rate': 0.5,
            },
        },
        'loss': {
            'losses': [
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
            ],
            'weights': [
                1,
            ],
        },
        'optimizer': {
            'name': 'Adam',
            'parameters': {
                'lr': 3e-3,
            },
        },
        'learning_rate_schedulers': [
            {
                'name': 'OneCycleLR',
                'parameters': {
                    'max_lr': 3e-3,
                    'total_steps': executions.NUMBER_EPOCHS * environment.math.ceil(environment.math.floor(102221 * 0.8) / (700)),
                    # total_steps = number_epochs * ceil(floor(data_samples * percentage_train) / batch_size)
                    'pct_start': 0.3,
                    'anneal_strategy': 'cos',
                    'div_factor': 25.0,
                    'final_div_factor': 1e4,
                },
                'monitor': None,
                'frequency': 'batch',
            },
        ],
        'metrics_class': {
            'metrics_classes': [
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
            ],
            'weights': [
                1,
            ],
        },
        'weight_init_type': '',
        'number_epochs': executions.NUMBER_EPOCHS,
        'bench_mark': False,
        'max_grad_norm': float('inf'),
        'metrics': [
            'false_negative',
            'false_positive',
            'true_negative',
            'true_positive',
            'accuracy',
        ],
        'objective': (
            'accuracy',
            'maximize',
            'validation',
        ),
    }
}

##### Execute Experiment 1, Execution 1: hidden dimension 256¶

In [ ]:
main.main()

#### Execution 2

Secondly, we will try it with a hidden dimension of 512.

##### Setup and configuration Experiment 1, Execution 2: hidden dimension 512

In [ ]:
executions.EXECUTIONS = {
    'experiment_1_execution_2_hidden_dim_512': {
        'data_transformation': {
            'name': 'mel_spectrogram_pre_saved',
            'parameters': {
                'sample_rate': 16000,
                'number_frames': 16000 * 29,
                'number_fft_points': 1024,
                'hop_length': 1024,
                'window_length': 512,
                'power': 2.0,
                'number_mel_channels': 128,
                'to_decibels': True,
            },
        },
        'data_split': {
            'name': 'random_split',
            'parameters': {
                'percentage_test': 0.2,
            },
        },
        'dataset':  {
            'name': 'dataset_with_spectograms_pre_saved',
            'parameters': {},
        },
        'dataloader': {
            'name': 'dataloader',
            'parameters': {
                'batch_size': 512,
                'shuffle': True,
                'pin_memory': False,
                'prefetch_factor': 5,
                'persistent_workers': True,
            },
        },
        'weights': {
            'parameters': {
                'method': 'inver_log',
                'epsilon': 1e-6,
            },
        },
        'model': {
            'name': 'baseline',
            'parameters': {
                'hidden_dim': 512,
                'classifier_hidden_layers': [],
                'threshold_method': 'roc_closest',
                'dropout_rate': 0.5,
            },
        },
        'loss': {
            'losses': [
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
            ],
            'weights': [
                1,
            ],
        },
        'optimizer': {
            'name': 'Adam',
            'parameters': {
                'lr': 3e-3,
            },
        },
        'learning_rate_schedulers': [
            {
                'name': 'OneCycleLR',
                'parameters': {
                    'max_lr': 3e-3,
                    'total_steps': executions.NUMBER_EPOCHS * environment.math.ceil(environment.math.floor(102221 * 0.8) / (512)),
                    # total_steps = number_epochs * ceil(floor(data_samples * percentage_train) / batch_size)
                    'pct_start': 0.3,
                    'anneal_strategy': 'cos',
                    'div_factor': 25.0,
                    'final_div_factor': 1e4,
                },
                'monitor': None,
                'frequency': 'batch',
            },
        ],
        'metrics_class': {
            'metrics_classes': [
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
            ],
            'weights': [
                1,
            ],
        },
        'weight_init_type': '',
        'number_epochs': executions.NUMBER_EPOCHS,
        'bench_mark': False,
        'max_grad_norm': float('inf'),
        'metrics': [
            'false_negative',
            'false_positive',
            'true_negative',
            'true_positive',
            'accuracy',
        ],
        'objective': (
            'accuracy',
            'maximize',
            'validation',
        ),
    }
}

##### Execute Experiment 1, Execution 2: hidden dimension 512¶

In [ ]:
main.main()

## Experiment 2

In this experiment we will try a custom CNN + classifier model with spectogram as input with hidden dim 128, trying to classify all genres at the same time.

#### Execution

##### Setup and configuration Experiment 2

In [ ]:
executions.EXECUTIONS = {
    'experiment_2_execution': {
        'data_transformation': {
            'name': 'mel_spectrogram_pre_saved',
            'parameters': {
                'sample_rate': 16000,
                'number_frames': 16000 * 29,
                'number_fft_points': 1024,
                'hop_length': 1024,
                'window_length': 512,
                'power': 2.0,
                'number_mel_channels': 128,
                'to_decibels': True,
            },
        },
        'data_split': {
            'name': 'random_split',
            'parameters': {
                'percentage_test': 0.2,
            },
        },
        'dataset':  {
            'name': 'dataset_with_spectograms_pre_saved',
            'parameters': {},
        },
        'dataloader': {
            'name': 'dataloader',
            'parameters': {
                'batch_size': 256,
                'shuffle': True,
                'pin_memory': False,
                'prefetch_factor': 5,
                'persistent_workers': True,
            },
        },
        'weights': {
            'parameters': {
                'method': 'inver_log',
                'epsilon': 1e-6,
            },
        },
        'model': {
            'name': 'full_cnn',
            'parameters': {
                'hidden_dim': 128,
                'classifier_hidden_layers': [],
                'threshold_method': 'roc_closest',
                'dropout_rate': 0.7,
            },
        },
        'loss': {
            'losses': [
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
            ],
            'weights': [
                1,
            ],
        },
        'optimizer': {
            'name': 'Adam',
            'parameters': {
                'lr': 3e-3,
            },
        },
        'learning_rate_schedulers': [
            {
                'name': 'OneCycleLR',
                'parameters': {
                    'max_lr': 3e-3,
                    'total_steps': executions.NUMBER_EPOCHS * environment.math.ceil(environment.math.floor(102221 * 0.8) / (256)), 
                    # total_steps = number_epochs * ceil(floor(data_samples * percentage_train) / batch_size)
                    'pct_start': 0.3,
                    'anneal_strategy': 'cos',
                    'div_factor': 25.0,
                    'final_div_factor': 1e4,
                },
                'monitor': None,
                'frequency': 'batch',
            },
        ],
        'metrics_class': {
            'metrics_classes': [
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
            ],
            'weights': [
                1,
            ],
        },
        'weight_init_type': '',
        'number_epochs': executions.NUMBER_EPOCHS,
        'bench_mark': False,
        'max_grad_norm': float('inf'),
        'metrics': [
            'false_negative',
            'false_positive',
            'true_negative',
            'true_positive',
            'accuracy',
        ],
        'objective': (
            'accuracy',
            'maximize',
            'validation',
        ),
    }
}

##### Execute Experiment 2¶

In [ ]:
main.main()

## Experiment 3

In this experiment we will try Fully Frozen ResNet + classifier models with spectogram as input, trying to classify all genres at the same time.

#### Execution 1

Firstly, we try with the ResNet18

##### Setup and configuration Experiment 3, Execution 1: ResNet18

In [ ]:
executions.EXECUTIONS = {
    'experiment_3_execution_1_res_net_18': {
        'data_transformation': {
            'name': 'mel_spectrogram_pre_saved',
            'parameters': {
                'sample_rate': 16000,
                'number_frames': 16000 * 29,
                'number_fft_points': 1024,
                'hop_length': 1024,
                'window_length': 512,
                'power': 2.0,
                'number_mel_channels': 128,
                'to_decibels': True,
            },
        },
        'data_split': {
            'name': 'random_split',
            'parameters': {
                'percentage_test': 0.2,
            },
        },
        'dataset':  {
            'name': 'dataset_with_spectograms_pre_saved',
            'parameters': {},
        },
        'dataloader': {
            'name': 'dataloader',
            'parameters': {
                'batch_size': 700,
                'shuffle': True,
                'pin_memory': False,
                'prefetch_factor': 5,
                'persistent_workers': True,
            },
        },
        'weights': {
            'parameters': {
                'method': 'inver_log',
                'epsilon': 1e-6,
            },
        },
        'model': {
            'name': 'res_net_18',
            'parameters': {
                'classifier_hidden_layers': [],
                'threshold_method': 'roc_closest',
                'dropout_rate': 0.7,
            },
        },
        'loss': {
            'losses': [
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
            ],
            'weights': [
                1,
            ],
        },
        'optimizer': {
            'name': 'Adam',
            'parameters': {
                'lr': 3e-3,
            },
        },
        'learning_rate_schedulers': [
            {
                'name': 'OneCycleLR',
                'parameters': {
                    'max_lr': 3e-3,
                    'total_steps': executions.NUMBER_EPOCHS * environment.math.ceil(environment.math.floor(102221 * 0.8) / (700)),
                    # total_steps = number_epochs * ceil(floor(data_samples * percentage_train) / batch_size)
                    'pct_start': 0.3,
                    'anneal_strategy': 'cos',
                    'div_factor': 25.0,
                    'final_div_factor': 1e4,
                },
                'monitor': None,
                'frequency': 'batch',
            },
        ],
        'metrics_class': {
            'metrics_classes': [
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
            ],
            'weights': [
                1,
            ],
        },
        'weight_init_type': '',
        'number_epochs': executions.NUMBER_EPOCHS,
        'bench_mark': False,
        'max_grad_norm': float('inf'),
        'metrics': [
            'false_negative',
            'false_positive',
            'true_negative',
            'true_positive',
            'accuracy',
        ],
        'objective': (
            'accuracy',
            'maximize',
            'validation',
        ),
    }
}

##### Execute Experiment 3, Execution 1: ResNet18¶

In [ ]:
main.main()

#### Execution 2

Secondly, we try with the ResNet50

##### Setup and configuration Experiment 3, Execution 2: ResNet50

In [ ]:
executions.EXECUTIONS = {
    'experiment_3_executions_2_res_net_50': {
        'data_transformation': {
            'name': 'mel_spectrogram_pre_saved',
            'parameters': {
                'sample_rate': 16000,
                'number_frames': 16000 * 29,
                'number_fft_points': 1024,
                'hop_length': 1024,
                'window_length': 512,
                'power': 2.0,
                'number_mel_channels': 128,
                'to_decibels': True,
            },
        },
        'data_split': {
            'name': 'random_split',
            'parameters': {
                'percentage_test': 0.2,
            },
        },
        'dataset':  {
            'name': 'dataset_with_spectograms_pre_saved',
            'parameters': {},
        },
        'dataloader': {
            'name': 'dataloader',
            'parameters': {
                'batch_size': 512,
                'shuffle': True,
                'pin_memory': False,
                'prefetch_factor': 5,
                'persistent_workers': True,
            },
        },
        'weights': {
            'parameters': {
                'method': 'inver_log',
                'epsilon': 1e-6,
            },
        },
        'model': {
            'name': 'res_net_50',
            'parameters': {
                'classifier_hidden_layers': [],
                'threshold_method': 'roc_closest',
                'dropout_rate': 0.7,
            },
        },
        'loss': {
            'losses': [
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
            ],
            'weights': [
                1,
            ],
        },
        'optimizer': {
            'name': 'Adam',
            'parameters': {
                'lr': 3e-3,
            },
        },
        'learning_rate_schedulers': [
            {
                'name': 'OneCycleLR',
                'parameters': {
                    'max_lr': 3e-3,
                    'total_steps': executions.NUMBER_EPOCHS * environment.math.ceil(environment.math.floor(102221 * 0.8) / (512)),
                    # total_steps = number_epochs * ceil(floor(data_samples * percentage_train) / batch_size)
                    'pct_start': 0.3,
                    'anneal_strategy': 'cos',
                    'div_factor': 25.0,
                    'final_div_factor': 1e4,
                },
                'monitor': None,
                'frequency': 'batch',
            },
        ],
        'metrics_class': {
            'metrics_classes': [
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
            ],
            'weights': [
                1,
            ],
        },
        'weight_init_type': '',
        'number_epochs': executions.NUMBER_EPOCHS,
        'bench_mark': False,
        'max_grad_norm': float('inf'),
        'metrics': [
            'false_negative',
            'false_positive',
            'true_negative',
            'true_positive',
            'accuracy',
        ],
        'objective': (
            'accuracy',
            'maximize',
            'validation',
        ),
    }
}

##### Execute Experiment 3, Execution 2: ResNet50¶

In [ ]:
main.main()

#### Execution 3

Thirdly, we try with the ResNet101

##### Setup and configuration Experiment 3, Execution 3: ResNet101

In [ ]:
executions.EXECUTIONS = {
    'experiment_3_executions_3_res_net_101': {
        'data_transformation': {
            'name': 'mel_spectrogram_pre_saved',
            'parameters': {
                'sample_rate': 16000,
                'number_frames': 16000 * 29,
                'number_fft_points': 1024,
                'hop_length': 1024,
                'window_length': 512,
                'power': 2.0,
                'number_mel_channels': 128,
                'to_decibels': True,
            },
        },
        'data_split': {
            'name': 'random_split',
            'parameters': {
                'percentage_test': 0.2,
            },
        },
        'dataset':  {
            'name': 'dataset_with_spectograms_pre_saved',
            'parameters': {},
        },
        'dataloader': {
            'name': 'dataloader',
            'parameters': {
                'batch_size': 512,
                'shuffle': True,
                'pin_memory': False,
                'prefetch_factor': 5,
                'persistent_workers': True,
            },
        },
        'weights': {
            'parameters': {
                'method': 'inver_log',
                'epsilon': 1e-6,
            },
        },
        'model': {
            'name': 'res_net_101',
            'parameters': {
                'classifier_hidden_layers': [],
                'threshold_method': 'roc_closest',
                'dropout_rate': 0.7,
            },
        },
        'loss': {
            'losses': [
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
            ],
            'weights': [
                1,
            ],
        },
        'optimizer': {
            'name': 'Adam',
            'parameters': {
                'lr': 3e-3,
            },
        },
        'learning_rate_schedulers': [
            {
                'name': 'OneCycleLR',
                'parameters': {
                    'max_lr': 3e-3,
                    'total_steps': executions.NUMBER_EPOCHS * environment.math.ceil(environment.math.floor(102221 * 0.8) / (512)),
                    # total_steps = number_epochs * ceil(floor(data_samples * percentage_train) / batch_size)
                    'pct_start': 0.3,
                    'anneal_strategy': 'cos',
                    'div_factor': 25.0,
                    'final_div_factor': 1e4,
                },
                'monitor': None,
                'frequency': 'batch',
            },
        ],
        'metrics_class': {
            'metrics_classes': [
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
            ],
            'weights': [
                1,
            ],
        },
        'weight_init_type': '',
        'number_epochs': executions.NUMBER_EPOCHS,
        'bench_mark': False,
        'max_grad_norm': float('inf'),
        'metrics': [
            'false_negative',
            'false_positive',
            'true_negative',
            'true_positive',
            'accuracy',
        ],
        'objective': (
            'accuracy',
            'maximize',
            'validation',
        ),
    }
}

##### Execute Experiment 3, Execution 3: ResNet101¶

In [ ]:
main.main()

## Experiment 4

In this experiment we will try Fine Tunning ResNet18 + classifier models with spectogram as input, trying to classify all genres at the same time.

#### Execution 1

Firstly, we try Fine Tunning the last layer only of the ResNet18 (last layer as the designers of ResNet18 defined it)

##### Setup and configuration Experiment 4, Execution 1: ResNet 18 Fine Tune Layer 4

In [ ]:
executions.EXECUTIONS = {
    'experiment_4_execution_1_layer_4': {
        'data_transformation': {
            'name': 'mel_spectrogram_pre_saved',
            'parameters': {
                'sample_rate': 16000,
                'number_frames': 16000 * 29,
                'number_fft_points': 1024,
                'hop_length': 1024,
                'window_length': 512,
                'power': 2.0,
                'number_mel_channels': 128,
                'to_decibels': True,
            },
        },
        'data_split': {
            'name': 'random_split',
            'parameters': {
                'percentage_test': 0.2,
            },
        },
        'dataset':  {
            'name': 'dataset_with_spectograms_pre_saved',
            'parameters': {},
        },
        'dataloader': {
            'name': 'dataloader',
            'parameters': {
                'batch_size': 700,
                'shuffle': True,
                'pin_memory': False,
                'prefetch_factor': 5,
                'persistent_workers': True,
            },
        },
        'weights': {
            'parameters': {
                'method': 'inver_log',
                'epsilon': 1e-6,
            },
        },
        'model': {
            'name': 'res_net_18_fine_tune_layer_4',
            'parameters': {
                'classifier_hidden_layers': [],
                'threshold_method': 'roc_closest',
                'dropout_rate': 0.7,
            },
        },
        'loss': {
            'losses': [
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
            ],
            'weights': [
                1,
            ],
        },
        'optimizer': {
            'name': 'Adam',
            'parameters': {
                'lr': 3e-3,
            },
        },
        'learning_rate_schedulers': [
            {
                'name': 'OneCycleLR',
                'parameters': {
                    'max_lr': 3e-3,
                    'total_steps': executions.NUMBER_EPOCHS * environment.math.ceil(environment.math.floor(102221 * 0.8) / (700)),
                    # total_steps = number_epochs * ceil(floor(data_samples * percentage_train) / batch_size)
                    'pct_start': 0.3,
                    'anneal_strategy': 'cos',
                    'div_factor': 25.0,
                    'final_div_factor': 1e4,
                },
                'monitor': None,
                'frequency': 'batch',
            },
        ],
        'metrics_class': {
            'metrics_classes': [
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
            ],
            'weights': [
                1,
            ],
        },
        'weight_init_type': '',
        'number_epochs': executions.NUMBER_EPOCHS,
        'bench_mark': False,
        'max_grad_norm': float('inf'),
        'metrics': [
            'false_negative',
            'false_positive',
            'true_negative',
            'true_positive',
            'accuracy',
        ],
        'objective': (
            'accuracy',
            'maximize',
            'validation',
        ),
    }
}

##### Execute Experiment 4, Execution 1: ResNet 18 Fine Tune Layer 4¶

In [ ]:
main.main()

#### Execution 2

Secondly, we try Fine Tunning the last block only of the last layer of the ResNet18 (last layer and last block as the designers of ResNet18 defined it)

##### Setup and configuration Experiment 4, Execution 2: ResNet 18 Fine Tune Last Block of Layer 4

In [ ]:
executions.EXECUTIONS = {
    'experiment_4_execution_2_layer_4_last_block': {
        'data_transformation': {
            'name': 'mel_spectrogram_pre_saved',
            'parameters': {
                'sample_rate': 16000,
                'number_frames': 16000 * 29,
                'number_fft_points': 1024,
                'hop_length': 1024,
                'window_length': 512,
                'power': 2.0,
                'number_mel_channels': 128,
                'to_decibels': True,
            },
        },
        'data_split': {
            'name': 'random_split',
            'parameters': {
                'percentage_test': 0.2,
            },
        },
        'dataset':  {
            'name': 'dataset_with_spectograms_pre_saved',
            'parameters': {},
        },
        'dataloader': {
            'name': 'dataloader',
            'parameters': {
                'batch_size': 700,
                'shuffle': True,
                'pin_memory': False,
                'prefetch_factor': 5,
                'persistent_workers': True,
            },
        },
        'weights': {
            'parameters': {
                'method': 'inver_log',
                'epsilon': 1e-6,
            },
        },
        'model': {
            'name': 'res_net_18_fine_tune_layer_4_last_block',
            'parameters': {
                'classifier_hidden_layers': [],
                'threshold_method': 'roc_closest',
                'dropout_rate': 0.7,
            },
        },
        'loss': {
            'losses': [
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
            ],
            'weights': [
                1,
            ],
        },
        'optimizer': {
            'name': 'Adam',
            'parameters': {
                'lr': 3e-3,
            },
        },
        'learning_rate_schedulers': [
            {
                'name': 'OneCycleLR',
                'parameters': {
                    'max_lr': 3e-3,
                    'total_steps': executions.NUMBER_EPOCHS * environment.math.ceil(environment.math.floor(102221 * 0.8) / (700)),
                    # total_steps = number_epochs * ceil(floor(data_samples * percentage_train) / batch_size)
                    'pct_start': 0.3,
                    'anneal_strategy': 'cos',
                    'div_factor': 25.0,
                    'final_div_factor': 1e4,
                },
                'monitor': None,
                'frequency': 'batch',
            },
        ],
        'metrics_class': {
            'metrics_classes': [
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
            ],
            'weights': [
                1,
            ],
        },
        'weight_init_type': '',
        'number_epochs': executions.NUMBER_EPOCHS,
        'bench_mark': False,
        'max_grad_norm': float('inf'),
        'metrics': [
            'false_negative',
            'false_positive',
            'true_negative',
            'true_positive',
            'accuracy',
        ],
        'objective': (
            'accuracy',
            'maximize',
            'validation',
        ),
    }
}

##### Execute Experiment 4, Execution 2: ResNet 18 Fine Tune Last Block of Layer 4¶

In [ ]:
main.main()

## Experiment 5

In this experiment we will try Fine Tunning ResNet18 + classifier models with spectogram as input, trying only the genres of top level.

#### Execution 1

Firstly, we try Fine Tunning the last layer only of the ResNet18 (last layer as the designers of ResNet18 defined it)

##### Setup and configuration Experiment 5, Execution 1: ResNet 18 Fine Tune Layer 4 Only Top Level Genres

In [ ]:
executions.EXECUTIONS = {
    'experiment_5_execution_1_layer_4_top_level': {
        'data_transformation': {
            'name': 'mel_spectrogram_pre_saved',
            'parameters': {
                'sample_rate': 16000,
                'number_frames': 16000 * 29,
                'number_fft_points': 1024,
                'hop_length': 1024,
                'window_length': 512,
                'power': 2.0,
                'number_mel_channels': 128,
                'to_decibels': True,
            },
        },
        'data_split': {
            'name': 'levels_split',
            'parameters': {
                'percentage_test': 0.2,
                'selected_levels': [0],
            },
        },
        'dataset':  {
            'name': 'dataset_with_spectograms_pre_saved',
            'parameters': {},
        },
        'dataloader': {
            'name': 'dataloader',
            'parameters': {
                'batch_size': 700,
                'shuffle': True,
                'pin_memory': False,
                'prefetch_factor': 5,
                'persistent_workers': True,
            },
        },
        'weights': {
            'parameters': {
                'method': 'inver_log',
                'epsilon': 1e-6,
            },
        },
        'model': {
            'name': 'res_net_18_fine_tune_layer_4',
            'parameters': {
                'classifier_hidden_layers': [],
                'threshold_method': 'roc_closest',
                'dropout_rate': 0.7,
            },
        },
        'loss': {
            'losses': [
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
            ],
            'weights': [
                1,
            ],
        },
        'optimizer': {
            'name': 'Adam',
            'parameters': {
                'lr': 3e-3,
            },
        },
        'learning_rate_schedulers': [
            {
                'name': 'OneCycleLR',
                'parameters': {
                    'max_lr': 3e-3,
                    'total_steps': executions.NUMBER_EPOCHS * environment.math.ceil(environment.math.floor(102221 * 0.8) / (700)),
                    # total_steps = number_epochs * ceil(floor(data_samples * percentage_train) / batch_size)
                    'pct_start': 0.3,
                    'anneal_strategy': 'cos',
                    'div_factor': 25.0,
                    'final_div_factor': 1e4,
                },
                'monitor': None,
                'frequency': 'batch',
            },
        ],
        'metrics_class': {
            'metrics_classes': [
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
            ],
            'weights': [
                1,
            ],
        },
        'weight_init_type': '',
        'number_epochs': executions.NUMBER_EPOCHS,
        'bench_mark': False,
        'max_grad_norm': float('inf'),
        'metrics': [
            'false_negative',
            'false_positive',
            'true_negative',
            'true_positive',
            'accuracy',
        ],
        'objective': (
            'accuracy',
            'maximize',
            'validation',
        ),
    }
}

##### Execute Experiment 5, Execution 1: ResNet 18 Fine Tune Layer 4 Only Top Level Genres¶

In [ ]:
main.main()

#### Execution 2

Secondly, we try Fine Tunning the last block only of the last layer of the ResNet18 (last layer and last block as the designers of ResNet18 defined it)

##### Setup and configuration Experiment 5, Execution 2: ResNet 18 Fine Tune Last Block of Layer 4 Only Top Level Genres

In [ ]:
executions.EXECUTIONS = {
    'experiment_5_execution_2_layer_4_last_block_top_level': {
        'data_transformation': {
            'name': 'mel_spectrogram_pre_saved',
            'parameters': {
                'sample_rate': 16000,
                'number_frames': 16000 * 29,
                'number_fft_points': 1024,
                'hop_length': 1024,
                'window_length': 512,
                'power': 2.0,
                'number_mel_channels': 128,
                'to_decibels': True,
            },
        },
        'data_split': {
            'name': 'levels_split',
            'parameters': {
                'percentage_test': 0.2,
                'selected_levels': [0],
            },
        },
        'dataset':  {
            'name': 'dataset_with_spectograms_pre_saved',
            'parameters': {},
        },
        'dataloader': {
            'name': 'dataloader',
            'parameters': {
                'batch_size': 700,
                'shuffle': True,
                'pin_memory': False,
                'prefetch_factor': 5,
                'persistent_workers': True,
            },
        },
        'weights': {
            'parameters': {
                'method': 'inver_log',
                'epsilon': 1e-6,
            },
        },
        'model': {
            'name': 'res_net_18_fine_tune_layer_4_last_block',
            'parameters': {
                'classifier_hidden_layers': [],
                'threshold_method': 'roc_closest',
                'dropout_rate': 0.7,
            },
        },
        'loss': {
            'losses': [
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
            ],
            'weights': [
                1,
            ],
        },
        'optimizer': {
            'name': 'Adam',
            'parameters': {
                'lr': 3e-3,
            },
        },
        'learning_rate_schedulers': [
            {
                'name': 'OneCycleLR',
                'parameters': {
                    'max_lr': 3e-3,
                    'total_steps': executions.NUMBER_EPOCHS * environment.math.ceil(environment.math.floor(102221 * 0.8) / (700)),
                    # total_steps = number_epochs * ceil(floor(data_samples * percentage_train) / batch_size)
                    'pct_start': 0.3,
                    'anneal_strategy': 'cos',
                    'div_factor': 25.0,
                    'final_div_factor': 1e4,
                },
                'monitor': None,
                'frequency': 'batch',
            },
        ],
        'metrics_class': {
            'metrics_classes': [
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
            ],
            'weights': [
                1,
            ],
        },
        'weight_init_type': '',
        'number_epochs': executions.NUMBER_EPOCHS,
        'bench_mark': False,
        'max_grad_norm': float('inf'),
        'metrics': [
            'false_negative',
            'false_positive',
            'true_negative',
            'true_positive',
            'accuracy',
        ],
        'objective': (
            'accuracy',
            'maximize',
            'validation',
        ),
    }
}

##### Execute Experiment 5, Execution 2: ResNet 18 Fine Tune Last Block of Layer 4 Only Top Level Genres¶

In [ ]:
main.main()

## Experiment 6

In this experiment we will try Fine Tunning the last block only of the last layer of the ResNet18 + classifier multihead with a head for each genre level, with spectogram as input, trying to classify all genres at the same time with diferent way to balance the positive weight of the classes in the loss.

#### Execution 1

Firstly, we try with same weight to all classes no matter what

##### Setup and configuration Experiment 6, Execution 1: Equal

In [ ]:
executions.EXECUTIONS = {
    'experiment_6_execution_1_multihead_equal': {
        'data_transformation': {
            'name': 'mel_spectrogram_pre_saved',
            'parameters': {
                'sample_rate': 16000,
                'number_frames': 16000 * 29,
                'number_fft_points': 1024,
                'hop_length': 1024,
                'window_length': 512,
                'power': 2.0,
                'number_mel_channels': 128,
                'to_decibels': True,
            },
        },
        'data_split': {
            'name': 'levels_split',
            'parameters': {
                'percentage_test': 0.2,
                'selected_levels': [],
            },
        },
        'dataset':  {
            'name': 'dataset_with_spectograms_pre_saved',
            'parameters': {},
        },
        'dataloader': {
            'name': 'dataloader',
            'parameters': {
                'batch_size': 700,
                'shuffle': True,
                'pin_memory': False,
                'prefetch_factor': 5,
                'persistent_workers': True,
            },
        },
        'weights': {
            'parameters': {
                'method': 'none',
                'epsilon': 1e-6,
            },
        },
        'model': {
            'name': 'res_net_18_multi_head',
            'parameters': {
                'classifier_hidden_layers': [],
                'threshold_method': 'roc_closest',
                'dropout_rate': 0.7,
            },
        },
       'loss': {
            'losses': [
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
            ],
            'weights': [
                1,
                1,
                1,
                1,
            ],
        },
        'optimizer': {
            'name': 'Adam',
            'parameters': {
                'lr': 3e-3,
            },
        },
        'learning_rate_schedulers': [
            {
                'name': 'OneCycleLR',
                'parameters': {
                    'max_lr': 3e-3,
                    'total_steps': executions.NUMBER_EPOCHS * environment.math.ceil(environment.math.floor(102221 * 0.8) / (700)),
                    # total_steps = number_epochs * ceil(floor(data_samples * percentage_train) / batch_size)
                    'pct_start': 0.3,
                    'anneal_strategy': 'cos',
                    'div_factor': 25.0,
                    'final_div_factor': 1e4,
                },
                'monitor': None,
                'frequency': 'batch',
            },
        ],
        'metrics_class': {
            'metrics_classes': [
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
            ],
            'weights': [
                1/4,
                1/4,
                1/4,
                1/4,
            ],
        },
        'weight_init_type': '',
        'number_epochs': executions.NUMBER_EPOCHS,
        'bench_mark': False,
        'max_grad_norm': float('inf'),
        'metrics': [
            'false_negative',
            'false_positive',
            'true_negative',
            'true_positive',
            'accuracy',
        ],
        'objective': (
            'accuracy',
            'maximize',
            'validation',
        ),
    }
}

##### Execute Experiment 6, Execution 1: Equal¶

In [ ]:
main.main()

#### Execution 2

Secondly, we try giving more weight to the classes with more positive appearances

##### Setup and configuration Experiment 6, Execution 2: More Appearance More Weight

In [ ]:
executions.EXECUTIONS = {
    'experiment_6_execution_2_multihead_ratio': {
        'data_transformation': {
            'name': 'mel_spectrogram_pre_saved',
            'parameters': {
                'sample_rate': 16000,
                'number_frames': 16000 * 29,
                'number_fft_points': 1024,
                'hop_length': 1024,
                'window_length': 512,
                'power': 2.0,
                'number_mel_channels': 128,
                'to_decibels': True,
            },
        },
        'data_split': {
            'name': 'levels_split',
            'parameters': {
                'percentage_test': 0.2,
                'selected_levels': [],
            },
        },
        'dataset':  {
            'name': 'dataset_with_spectograms_pre_saved',
            'parameters': {},
        },
        'dataloader': {
            'name': 'dataloader',
            'parameters': {
                'batch_size': 600,
                'shuffle': True,
                'pin_memory': False,
                'prefetch_factor': 5,
                'persistent_workers': True,
            },
        },
        'weights': {
            'parameters': {
                'method': 'ratio',
                'epsilon': 1e-6,
            },
        },
        'model': {
            'name': 'res_net_18_multi_head',
            'parameters': {
                'classifier_hidden_layers': [],
                'threshold_method': 'roc_closest',
                'dropout_rate': 0.7,
            },
        },
       'loss': {
            'losses': [
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
            ],
            'weights': [
                1,
                1,
                1,
                1,
            ],
        },
        'optimizer': {
            'name': 'Adam',
            'parameters': {
                'lr': 3e-3,
            },
        },
        'learning_rate_schedulers': [
            {
                'name': 'OneCycleLR',
                'parameters': {
                    'max_lr': 3e-3,
                    'total_steps': executions.NUMBER_EPOCHS * environment.math.ceil(environment.math.floor(102221 * 0.8) / (600)),
                    # total_steps = number_epochs * ceil(floor(data_samples * percentage_train) / batch_size)
                    'pct_start': 0.3,
                    'anneal_strategy': 'cos',
                    'div_factor': 25.0,
                    'final_div_factor': 1e4,
                },
                'monitor': None,
                'frequency': 'batch',
            },
        ],
        'metrics_class': {
            'metrics_classes': [
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
            ],
            'weights': [
                1/4,
                1/4,
                1/4,
                1/4,
            ],
        },
        'weight_init_type': '',
        'number_epochs': executions.NUMBER_EPOCHS,
        'bench_mark': False,
        'max_grad_norm': float('inf'),
        'metrics': [
            'false_negative',
            'false_positive',
            'true_negative',
            'true_positive',
            'accuracy',
        ],
        'objective': (
            'accuracy',
            'maximize',
            'validation',
        ),
    }
}

##### Execute Experiment 6, Execution 2: More Appearance More Weight

In [ ]:
main.main()

#### Execution 3

Thirdly, we try giving more weight to the classes with less positive appearances

##### Setup and configuration Experiment 6, Execution 3: Less Appearance More Weight

In [ ]:
executions.EXECUTIONS = {
    'experiment_6_execution_3_multihead_inv_log': {
        'data_transformation': {
            'name': 'mel_spectrogram_pre_saved',
            'parameters': {
                'sample_rate': 16000,
                'number_frames': 16000 * 29,
                'number_fft_points': 1024,
                'hop_length': 1024,
                'window_length': 512,
                'power': 2.0,
                'number_mel_channels': 128,
                'to_decibels': True,
            },
        },
        'data_split': {
            'name': 'levels_split',
            'parameters': {
                'percentage_test': 0.2,
                'selected_levels': [],
            },
        },
        'dataset':  {
            'name': 'dataset_with_spectograms_pre_saved',
            'parameters': {},
        },
        'dataloader': {
            'name': 'dataloader',
            'parameters': {
                'batch_size': 600,
                'shuffle': True,
                'pin_memory': False,
                'prefetch_factor': 5,
                'persistent_workers': True,
            },
        },
        'weights': {
            'parameters': {
                'method': 'inver_log',
                'epsilon': 1e-6,
            },
        },
        'model': {
            'name': 'res_net_18_multi_head',
            'parameters': {
                'classifier_hidden_layers': [],
                'threshold_method': 'roc_closest',
                'dropout_rate': 0.7,
            },
        },
       'loss': {
            'losses': [
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
                {
                    'name': 'BCEWithLogitsLoss',
                    'parameters': {
                        'reduction': 'mean',
                    },
                },
            ],
            'weights': [
                1,
                1,
                1,
                1,
            ],
        },
        'optimizer': {
            'name': 'Adam',
            'parameters': {
                'lr': 3e-3,
            },
        },
        'learning_rate_schedulers': [
            {
                'name': 'OneCycleLR',
                'parameters': {
                    'max_lr': 3e-3,
                    'total_steps': executions.NUMBER_EPOCHS * environment.math.ceil(environment.math.floor(102221 * 0.8) / (600)),
                    # total_steps = number_epochs * ceil(floor(data_samples * percentage_train) / batch_size)
                    'pct_start': 0.3,
                    'anneal_strategy': 'cos',
                    'div_factor': 25.0,
                    'final_div_factor': 1e4,
                },
                'monitor': None,
                'frequency': 'batch',
            },
        ],
        'metrics_class': {
            'metrics_classes': [
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
                {
                    'name': 'multi_label_metrics',
                    'parameters': {},
                },
            ],
            'weights': [
                1/4,
                1/4,
                1/4,
                1/4,
            ],
        },
        'weight_init_type': '',
        'number_epochs': executions.NUMBER_EPOCHS,
        'bench_mark': False,
        'max_grad_norm': float('inf'),
        'metrics': [
            'false_negative',
            'false_positive',
            'true_negative',
            'true_positive',
            'accuracy',
        ],
        'objective': (
            'accuracy',
            'maximize',
            'validation',
        ),
    }
}

##### Execute Experiment 6, Execution 3: Less Appearance More Weight

In [ ]:
main.main()

# Finish execution and close eveything

In [ ]:
environment.finish()
utils.collect_memory()
%reset -sf